<h1>This is our simple demo file</h1>
<h4>Running all cells will compute our approach for CIFAR-10 or MNIST (which will be downloaded)</h4>
This code (except with modifications to use torch lightning) has been used to obtain all of our benchmarks as specified in the paper. We keep it in one simple Jupyter Notebook to make sure it stays accessible<br/>
(and because "we can't code properly", according to a dear co-author).<br/>
<u>At some point, we will turn this into a proper git repository</u>

--> follow https://github.com/DaiDaiLoh/QG-VAE for the prettier version!

<h1>!!! First few epochs use very few pseudo frequencies and can be hence <b>dreadful</b> to look at!!!</h1>
We can only split Voronoi cells aka codewords that are in use, hence it takes a bit to use the full codebook. Improving that would do a lot - but we beat benchmarks anyway, so we left it like this for now.

In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import datasets, transforms
import time
import torch.nn as nn
import math
import numpy as np
import platform
import sys
import os
import matplotlib.pyplot as plt
import torchvision

torch.set_printoptions(precision=8, sci_mode=False)

In [ ]:
def mkdir(path):
    try:
        os.makedirs(path)
    except:
        pass

In [ ]:
class Settings:
    ### SETTINGS FOR VQVAE ###
    VQVAE_D = 64 #dimensionality in which we cluster
    VQVAE_K = 512 #number of different codewords (i.e. number of voronoi cells/clusters)
    VQVAE_C = 8*8 #number of codewords to describe one encoded item
    DS_SPEED_FACTOR = 0 #number of times we downscale our feature map; called "f" in the paper
    USE_DROPOUT = False #if we want to order our latent code from important to unimportant 

    ### SETTINGS FOR NETWORKS ###
    CHANNELS_MAXIMUM = 128 #maximum number of channels in our U-Nets
    CHANNELS_MINIMUM = 16 #minimum number of channels we output in U-Nets
    NUM_WORKERS = 0 #for linux, put to something higher, but windows does not like this for some reason
    RES_BLOCKS = 1
    DIM_L_EMBED = 16
    DO_QUANTISE = True
    NUM_HEADS = 8

    NUM_TRAIN_BATCHES_UNTIL_RESET = 100 #reset unused entries every 100 epochs

    ### RUN SETTINGS ###
    DATASET = ["CIFAR", "MNIST"][0]

    CLUSTERRUN = False

    RUN_NAME = "" #is set later automatically

    USE_CHECKPOINTS = False

SETTINGS = Settings()

In [ ]:
SETTINGS.RUN_NAME = SETTINGS.DATASET + ("_dropout" if SETTINGS.USE_DROPOUT else "")
print("RUN NAME: ",SETTINGS.RUN_NAME)

In [ ]:
mkdir("outputs/")
mkdir("outputs/"+SETTINGS.RUN_NAME+"/")

In [ ]:
assert(not SETTINGS.USE_DROPOUT or SETTINGS.USE_GLOBAL)

In [ ]:
if SETTINGS.DATASET == "CIFAR":
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])
    trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    trainloader = DataLoader(trainset, batch_size=32, shuffle=True, num_workers=SETTINGS.NUM_WORKERS)
    testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    testloader = DataLoader(testset, batch_size=32, shuffle=False, num_workers=SETTINGS.NUM_WORKERS)
elif SETTINGS.DATASET == "MNIST":
    transform = transforms.Compose([transforms.ToTensor()])
    trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    trainloader = DataLoader(trainset, batch_size=32, shuffle=True, num_workers=SETTINGS.NUM_WORKERS)
    testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
    testloader = DataLoader(testset, batch_size=32, shuffle=False, num_workers=SETTINGS.NUM_WORKERS)
else:
    print("INVALID DATASET CHOSEN!")
    assert(False)

def imshow(img):
    #check if is in interactive session:
    if 'ipykernel' in sys.modules:
        npimg = img.clamp(0.0, 1.0).numpy()
        plt.imshow(np.transpose(npimg, (1, 2, 0)))
        plt.show()

def save_image(img, name):
    torchvision.utils.save_image(img, name)

for images, labels in trainloader:
    print("IMAGE VALUE RANGES: ",images.min(), " to ",images.max())
    print("IMAGE SHAPE: ",images.shape)
    imshow(torchvision.utils.make_grid(images))
    save_image(torchvision.utils.make_grid(images), "outputs/"+SETTINGS.RUN_NAME+"/test.png")
    break

In [ ]:
def positional_encoding(x, DIM_L_EMBED):
    rets = []
    for i in range(DIM_L_EMBED):
        for fn in [torch.sin, torch.cos]:
            rets.append(fn((2. ** i) * x))
    return torch.cat(rets, -1)

In [ ]:
assert(SETTINGS.VQVAE_C % SETTINGS.NUM_HEADS == 0)

In [ ]:
SETTINGS.TRAIN_BATCHES = len(trainloader)
SETTINGS.TEST_BATCHES = len(testloader)

In [ ]:
for images, labels in trainloader:
    SETTINGS.INPUT_C = images.size()[1]
    SETTINGS.INPUT_W = images.size()[2]
    break
SETTINGS.INPUT_H = SETTINGS.INPUT_W

In [ ]:
class SmallResidualBlock(nn.Module):
    #basically, this is the Imagen residual Block
    def __init__(self, c_in, c_int, c_out, res_dims, relu_slope=0.01):
        super(SmallResidualBlock, self).__init__()

        n1_no_grp = 16
        n2_no_grp = 16
        while (c_in) % n1_no_grp != 0:
            n1_no_grp -= 1
        while (c_int) % n2_no_grp != 0:
            n2_no_grp -= 1
        assert(n1_no_grp > 0 and n2_no_grp > 0)
        self.norm1 = nn.GroupNorm(num_groups=n1_no_grp, num_channels=c_in, eps=1e-05, affine=True)
        self.norm2 = nn.GroupNorm(num_groups=n2_no_grp, num_channels=c_int, eps=1e-05, affine=True)

        self.conv_res = nn.Conv2d(kernel_size=1, in_channels=c_in, out_channels=c_out, stride=1, padding=0, bias=False)
        self.conv_res_conditional = nn.Conv2d(kernel_size=1, in_channels=res_dims, out_channels=c_in, stride=1, padding=0, bias=False)
        self.conv1 = nn.Conv2d(kernel_size=3, in_channels=c_in, out_channels=c_int, stride=1, padding=1)
        self.conv2 = nn.Conv2d(kernel_size=3, in_channels=c_int, out_channels=c_out, stride=1, padding=1)
        
        self.relu = nn.LeakyReLU(negative_slope=relu_slope)
        self.activation = nn.SiLU(inplace=False)

    def forward(self, x, x_conditional_res):
        x_res = self.conv_res(x)

        x = x + self.conv_res_conditional(x_conditional_res)
        #according to Google's Imagen:
        x = self.activation(self.norm1(x))
        x = self.activation(self.norm2(self.conv1(x)))
        x = self.conv2(x)
        x = x + x_res
        return x
    
class ResidualBlocks(nn.Module):
    def __init__(self, c_in, c_int, c_out, res_dims, relu_slope=0.01, RES_BLOCKS=3):
        super(ResidualBlocks, self).__init__()

        #list of small residual blocks #
        self.blocks = nn.ModuleList()
        # use exactly THREE res blocks!
        for i in range(0, RES_BLOCKS):
            cur_c_in, cur_c_out = c_int, c_int
            if i == 0:
                cur_c_in = c_in
            if i == RES_BLOCKS - 1:
                cur_c_out = c_out
            self.blocks.append(SmallResidualBlock(cur_c_in, c_int, cur_c_out, res_dims=res_dims, relu_slope=relu_slope))

    def forward(self, x, x_conditional_res):
        for block in self.blocks:
            x = block(x, x_conditional_res)
        return x

class UNetBlock(nn.Module):
    def __init__(self, SETTINGS, c_in, c_int, width_height, NO_CHANNELS_MAX, NO_CHANNELS_MIN, res_dims, relu_slope=0.01, current_layer=0, skip_up_layers=0, skip_down_layers=0, target_out=None):
        super(UNetBlock, self).__init__()

        self.in_conv = nn.Conv2d(kernel_size=3, in_channels=c_in, out_channels=c_int, stride=1, padding=1)
        c_next = min(c_int * 2, NO_CHANNELS_MAX)
        self.in_res = ResidualBlocks(c_int, c_int, c_out=c_next, res_dims=res_dims, relu_slope=relu_slope, RES_BLOCKS=SETTINGS.RES_BLOCKS)

        self.width_height = width_height

        if self.width_height > 8 or (skip_up_layers > current_layer or current_layer < skip_down_layers): #don't go below 8x8, that makes no sense (except if we have to)
            self.unet = UNetBlock(SETTINGS, c_in=c_next, c_int=c_next, width_height=int(width_height/2), NO_CHANNELS_MAX=NO_CHANNELS_MAX, NO_CHANNELS_MIN=NO_CHANNELS_MIN, res_dims=res_dims, relu_slope=relu_slope, current_layer=current_layer+1, skip_up_layers=skip_up_layers, skip_down_layers=skip_down_layers)
            self.downsample = nn.Upsample(scale_factor=0.5, mode='bilinear')
            self.downsample_cond = nn.Upsample(scale_factor=0.5, mode='nearest')
            self.upsample = nn.Upsample(scale_factor=2, mode='bilinear')

            next_input = self.unet.out_conv.out_channels
        else:
            self.unet = None
            next_input = c_next

        dim_out = c_int
        if skip_up_layers > current_layer:
            dim_out = self.unet.out_conv.out_channels
        
        if current_layer == 0 and target_out != None:
            dim_out = target_out
        
        self.out_res = ResidualBlocks(next_input, max(c_int, dim_out), c_out=dim_out, res_dims=res_dims, relu_slope=relu_slope, RES_BLOCKS=SETTINGS.RES_BLOCKS)
        self.out_conv = nn.Conv2d(kernel_size=3, in_channels=dim_out, out_channels=dim_out, stride=1, padding=1)
        self.full_residual = nn.Conv2d(kernel_size=1, in_channels=c_in, out_channels=dim_out, stride=1, padding=0)
        
        if current_layer < skip_down_layers:
            #make sure we do SHRINK the number of parameters!
            if self.unet == None:
                prev_ch = c_next
            else:
                prev_ch = int(self.unet.out_conv.out_channels)
            layers_to_go = min(NO_CHANNELS_MIN * (2 ** current_layer), prev_ch)
            if current_layer == 0 and target_out != None:
                layers_to_go = target_out
            
            self.out_res = ResidualBlocks(next_input, next_input, c_out=layers_to_go, res_dims=res_dims, relu_slope=relu_slope, RES_BLOCKS=SETTINGS.RES_BLOCKS)
            self.out_conv = nn.Conv2d(kernel_size=3, in_channels=layers_to_go, out_channels=layers_to_go, stride=1, padding=1)
            self.full_residual = nn.Conv2d(kernel_size=1, in_channels=c_in, out_channels=layers_to_go, stride=1, padding=0)
        
        self.activation = nn.SiLU(inplace=False)

        self.current_layer = current_layer
        self.skip_up_layers = skip_up_layers
        self.skip_down_layers = skip_down_layers

    def forward(self, x, residual):
        #input block
        x = self.in_conv(x)

        #apply (multiple?) residual blocks
        x = self.in_res(x, residual)
        if x.size()[2] > 1 and self.unet != None:
            #downscale
            if self.skip_down_layers <= self.current_layer:
                x = self.downsample(x)
            ds_res = residual
            while x.size()[2] < ds_res.size()[2]:
                ds_res = self.downsample_cond(ds_res)
            #recursive
            x = self.unet(x, ds_res)
            #upscale
            if self.skip_up_layers <= self.current_layer:
                x = self.upsample(x)
                
        #output residual
        while x.size()[2] > residual.size()[2]:
            residual = self.upsample(residual)
        while x.size()[2] < residual.size()[2]:
            residual = self.downsample(residual)

        x = self.out_res(x, residual)
        #convolution
        x = self.out_conv(x)

        return x

class UNet(nn.Module):
    def __init__(self, SETTINGS, c_in, c_int, c_out, NO_CHANNELS_MAX, NO_CHANNELS_MIN, width_height, skip_up_layers=0, skip_down_layers=0):
        super(UNet, self).__init__()

        assert(skip_up_layers == 0 or skip_down_layers == 0)

        self.SETTINGS = SETTINGS
        res_dims = 2 * 2 * self.SETTINGS.DIM_L_EMBED + c_in
        self.position = nn.ParameterList()
        wh = width_height
        self.wh = width_height
        while wh >= 1:
            self.position.append(nn.Parameter(self.grid_positional_encoding(wh)[None], requires_grad=False))
            wh = int(wh / 2)
        
        self.unet_blocks = UNetBlock(SETTINGS, c_in, c_int, width_height=width_height, NO_CHANNELS_MAX=NO_CHANNELS_MAX, NO_CHANNELS_MIN=NO_CHANNELS_MIN, res_dims=res_dims, skip_up_layers=skip_up_layers, skip_down_layers=skip_down_layers, target_out=c_out)

        self.downsample = nn.Upsample(scale_factor=0.5, mode='bilinear')
        self.upsample = nn.Upsample(scale_factor=2.0, mode='bilinear')
        
        self.mega_residual = torch.nn.Conv2d(kernel_size=3, in_channels=c_in, out_channels=self.unet_blocks.out_conv.out_channels, stride=1, padding=1, bias=False)

        self.out_projection = nn.Conv2d(kernel_size=3, in_channels=self.unet_blocks.out_conv.out_channels, out_channels=c_out, stride=1, padding=1, bias=False)
        
    def grid_positional_encoding(self, width_height):
        grid = torch.ones(width_height, width_height, 2)
        for x in range(0, width_height):
            grid[x,:,0] = x / width_height
        for y in range(0, width_height):
            grid[:,y,1] = y / width_height
        #first dimension is what we should encode
        rets = []
        for i in range(self.SETTINGS.DIM_L_EMBED):
            for fn in [torch.sin, torch.cos]:
                rets.append(fn((2. ** i) * grid))
        return torch.cat(rets, -1).transpose(2,1).transpose(1,0)

    #decode in the sense of: turn into embeddings
    def decode_indices(self, indices, codebook):
        w, h = indices.size()[1], indices.size()[2]
        output = codebook.transpose(0,1)[indices.view(-1)]
        output = output.transpose(0,1)
        output = output.view(output.size()[0], indices.size()[0], w, h).transpose(0,1)
        return output
    
    def get_positions(self, width_height):
        positions = []
        cur_wh = width_height
        while cur_wh >= 1:
            ### use only relative position
            diff = self.wh - cur_wh
            if diff == 0:
                index = 0
            else:
                index = int(math.log2(self.wh)) - int(math.log2(cur_wh)) #0 at first
            position_absolute = self.position[index].clone()
            cur_wh = int(cur_wh / 2)
            positions.append(position_absolute)
        return positions
        
    def forward(self, x):
        residual = torch.cat((x, self.position[0].repeat(x.size()[0], 1, 1, 1)), 1)
        mega_res = self.mega_residual(x)
        #downscale:
        out = self.unet_blocks(x, residual)

        while out.size()[2] > mega_res.size()[2]:
            mega_res = self.upsample(mega_res)
        while out.size()[2] < mega_res.size()[2]:
            mega_res = self.downsample(mega_res)
        result = self.out_projection(out + mega_res)
        return result

In [ ]:
class ConvEncoder(nn.Module):
    def __init__(self, c_in):
        super(ConvEncoder, self).__init__()
        self.relu = torch.nn.LeakyReLU()
        self.downsample = nn.Upsample(scale_factor=0.5, mode='bilinear')

        self.encoder = nn.Sequential()
        self.encoder.append(torch.nn.Conv2d(in_channels=c_in, out_channels=SETTINGS.CHANNELS_MAXIMUM, kernel_size=3, stride=1, padding=1))
        self.encoder.append(self.relu)
        self.encoder.append(self.downsample)
        self.encoder.append(torch.nn.Conv2d(in_channels=SETTINGS.CHANNELS_MAXIMUM, out_channels=SETTINGS.CHANNELS_MAXIMUM, kernel_size=3, stride=1, padding=1))
        self.encoder.append(self.relu)
        self.encoder.append(torch.nn.Conv2d(in_channels=SETTINGS.CHANNELS_MAXIMUM, out_channels=SETTINGS.CHANNELS_MAXIMUM, kernel_size=3, stride=1, padding=1))
        self.encoder.append(self.relu)
        self.encoder.append(self.downsample)
        self.encoder.append(torch.nn.Conv2d(in_channels=SETTINGS.CHANNELS_MAXIMUM, out_channels=SETTINGS.VQVAE_C, kernel_size=3, stride=1, padding=1))
        self.encoder.append(self.relu)
        
    def forward(self, x):
        return self.encoder(x)

class ConvDecoder(nn.Module):
    def __init__(self, c_out):
        super(ConvDecoder, self).__init__()
        self.relu = torch.nn.LeakyReLU()
        self.upsample = nn.Upsample(scale_factor=2.0, mode='bilinear')

        self.decoder = nn.Sequential()
        self.decoder.append(torch.nn.Conv2d(in_channels=SETTINGS.VQVAE_C, out_channels=SETTINGS.CHANNELS_MAXIMUM, kernel_size=3, stride=1, padding=1))
        self.decoder.append(self.relu)
        self.decoder.append(torch.nn.Conv2d(in_channels=SETTINGS.CHANNELS_MAXIMUM, out_channels=SETTINGS.CHANNELS_MAXIMUM, kernel_size=3, stride=1, padding=1))
        self.decoder.append(self.relu)
        self.decoder.append(self.upsample)
        self.decoder.append(torch.nn.Conv2d(in_channels=SETTINGS.CHANNELS_MAXIMUM, out_channels=SETTINGS.CHANNELS_MAXIMUM, kernel_size=3, stride=1, padding=1))
        self.decoder.append(self.relu)
        self.decoder.append(self.upsample)
        self.decoder.append(torch.nn.Conv2d(in_channels=SETTINGS.CHANNELS_MAXIMUM, out_channels=c_out, kernel_size=3, stride=1, padding=1))
        self.decoder.append(self.relu) #only because we have the output values in [0, 1]
    
    def forward(self, x):
        return self.decoder(x)

class Discriminator(nn.Module):
    def __init__(self, SETTINGS, c_in, c_int, width_height, input_channels):
        super(Discriminator, self).__init__()

        #c_in, c_int, c_out, res_dims=0, relu_slope=0.01
        self.block_in = SmallResidualBlock(c_in, c_int, c_int, res_dims=c_in, relu_slope=0.2)
        self.unet_blocks = UNetBlock(SETTINGS, c_int, c_int, c_int, width_height=width_height, NO_CHANNELS_MAX=c_int, NO_CHANNELS_MIN=16, res_dims=input_channels, relu_slope=0.2)
        self.block_out = SmallResidualBlock(c_int, c_int, 1, res_dims=c_in, relu_slope=0.2)
        
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x_input = x
        #in
        x = self.block_in(x, x_input)
        #unet
        x = self.unet_blocks(x, x_input)
        #residual after input layer
        x = self.block_out(x, x_input)
        #patchwise discriminator
        return x

In [ ]:
def get_fullsize(x):
    if len(x.size()) == 1:
        return "ERROR: get_fullsize() called with 1D tensor"
    elif len(x.size()) == 2:
        return x.size()[1]
    elif len(x.size()) == 3:
        return x.size()[1]*x.size()[2]
    elif len(x.size()) == 4:
        return x.size()[1]*x.size()[2]*x.size()[3]
    elif len(x.size()) == 5:
        return x.size()[1]*x.size()[2]*x.size()[3]*x.size()[4]
    else:
        print("ERROR: get_fullsize() called with tensor of size ", x.size())

class BaseAE(nn.Module):
    def __init__(self, SETTINGS, c_in):
        super(BaseAE, self).__init__()

        self.SETTINGS = SETTINGS
        self.relu = torch.nn.LeakyReLU() #we use leaky relu as activation function, as it also has a gradient for an input < 0
        self.reduce = torch.nn.AvgPool2d(kernel_size=2, stride=2, padding=0)
        self.upscale = torch.nn.Upsample(scale_factor=2.0, mode='bilinear')

        self.residual_encoder = torch.nn.Conv2d(kernel_size=1, in_channels=SETTINGS.INPUT_C, out_channels=SETTINGS.VQVAE_C, stride=1, padding=0)
        self.residual_decoder = torch.nn.Conv2d(kernel_size=1, in_channels=SETTINGS.VQVAE_C, out_channels=SETTINGS.INPUT_C, stride=1, padding=0)

        self.encoder = UNet(SETTINGS, c_in=c_in, c_int=SETTINGS.CHANNELS_MINIMUM, c_out=SETTINGS.VQVAE_C, NO_CHANNELS_MAX=SETTINGS.CHANNELS_MAXIMUM, NO_CHANNELS_MIN=SETTINGS.CHANNELS_MINIMUM, width_height=SETTINGS.INPUT_W, skip_up_layers=SETTINGS.DS_SPEED_FACTOR)
        
        #create a list of projections:
        self.down_proj = nn.ModuleList()
        self.up_proj   = nn.ModuleList()
        for i in range(0, SETTINGS.NUM_HEADS):
            self.down_proj.append(torch.nn.Conv1d(in_channels=int(SETTINGS.INPUT_W*SETTINGS.INPUT_H/(2 ** SETTINGS.DS_SPEED_FACTOR)/(2 ** SETTINGS.DS_SPEED_FACTOR)), out_channels=SETTINGS.VQVAE_D, kernel_size=1, stride=1, padding=0))
            self.up_proj.append(torch.nn.Conv1d(in_channels=SETTINGS.VQVAE_D, out_channels=int(SETTINGS.INPUT_W*SETTINGS.INPUT_H/(2 ** SETTINGS.DS_SPEED_FACTOR)/(2 ** SETTINGS.DS_SPEED_FACTOR)), kernel_size=1, stride=1, padding=0))
        self.decoder = UNet(SETTINGS, c_in=SETTINGS.VQVAE_C, c_int=SETTINGS.CHANNELS_MAXIMUM, c_out=c_in, NO_CHANNELS_MAX=SETTINGS.CHANNELS_MAXIMUM, NO_CHANNELS_MIN=SETTINGS.CHANNELS_MINIMUM, width_height=int(SETTINGS.INPUT_W/(2 ** SETTINGS.DS_SPEED_FACTOR)), skip_down_layers=SETTINGS.DS_SPEED_FACTOR)

        #use individual codebooks per pseudo frequency:
        self.codebook = torch.nn.Parameter((torch.rand(SETTINGS.VQVAE_C, SETTINGS.VQVAE_K, SETTINGS.VQVAE_D) * 2.0 - 1.0) * 0.5, requires_grad=True)
        self.offset = torch.nn.Parameter((torch.rand(SETTINGS.VQVAE_D, SETTINGS.VQVAE_C) * 2.0 - 1.0) * 0.5, requires_grad=True)

        self.pos = positional_encoding(torch.arange(0, SETTINGS.VQVAE_C)[:,None] / (SETTINGS.VQVAE_C+1), 10)[None,:,:].transpose(1,2)

    def quantise_individually(self, x):
        x_before_rounding = x.clone()
        #size  x in: [b x D x C x 1]
        #size cb in: [C x K x D]
        x_before_rounding = x.clone()

        ### torch.rand(C, D, K)
        B = x.size()[0]
        x = x.transpose(1,2)[:,:,:,0]
        codebook = self.codebook.transpose(1,2)

        dists = (x[..., None] - codebook[None]).square().sum(-2)#
        idx = dists.argmin(-1)
        
        codebook_expanded = codebook[None].expand(B, -1, -1, -1)
        idx_expanded = idx[..., None, None].expand(-1, -1, self.SETTINGS.VQVAE_D, -1)
        
        x_rounded = torch.gather(codebook_expanded, -1, idx_expanded).transpose(1,2)

        loss_commitment = (x_before_rounding - x_rounded.detach()).square().mean() #only have gradient for encoder 
        loss_codebook   = (x_before_rounding.detach() - x_rounded).square().mean() #only have gradient for codebook

        x = x_before_rounding - (x_before_rounding - x_rounded).detach()
        
        return x, loss_codebook, loss_commitment, idx.view(x.size()[0], x.size()[2], x.size()[3])
    
    def quantise(self, x):
        #this is where the magic happens:
        #    1. take input of size [b x W x H x VQVAE_D] and reshape to [b x VQVAE_D x W x H ];
        #       this means we have ALL the codewords in the last dimension now, so we can just merge together all other dimensions to easily compare them to the codewords
        x = x.permute(0, 2, 3, 1).contiguous()
        size_in = x.size()
        x = x.view(-1, self.SETTINGS.VQVAE_D)
        #   2. compare [b*W*H x VQVAE_D] to [CODEWORDS x VQVAE_D]: by re-shaping to have [b*W*H x 1 x VQVAE_D] and [1 x CODEWORDS x VQVAE_D], the substraction will
        #      give us a [b*W*H x CODEWORDS X VQVAE_D] tensor of all distance pairs between input elements (b*W*H) and the CODEWORDS-many codewords; just square, 
        #      then sum up the last dimension to get sum of the distance values for each dimension (=euclidean squared distance)
        #      and then take the argmin to get the index of the closest codeword for each input element - these are our indices we want!
        if self.SETTINGS.DATASET != "IMAGENET":
            with torch.no_grad():
                indices = (x[:,None] - self.codebook.view(1, self.SETTINGS.VQVAE_K, self.SETTINGS.VQVAE_D)).square().sum(dim=2).argmin(dim=1)
        else:
            #quantise in chunks
            chunksize = int(x.size()[0]/8)
            with torch.no_grad():
                index_begin = 0
                index_end   = chunksize
                acc_indices = []
                while index_begin < x.size()[0]:
                    indices = (x[index_begin:index_end][:,None] - self.codebook.view(1, self.SETTINGS.VQVAE_K, self.SETTINGS.VQVAE_D)).square().sum(dim=2).argmin(dim=1)
                    acc_indices.append(indices)
                    index_begin += chunksize
                    index_end += chunksize
                indices = torch.cat(acc_indices)
            
        #   3. we take those indices we just looked up: these are the indices of the closest codeword for each input element, so we can now look up the actual 
        #      codeword to find the closest codeword to each input element aka the "centroids" we round to
        x_rounded = self.codebook[indices]
        
        #   4. we calculate the loss for the commitment loss (i.e. does the encoder produce stuff from the codebook?) and the codebook loss (how close is the codebook to the encoder outputs?)
        #      the value is exactly the same for both terms, but this way we can a) train encoder and codebook with different magnitudes and b) the encoder outputs and codebook outputs are
        #      not just "shrinked" to one super tiny value: the loss we be almost zero if codebook and encoder outputs would be just downscaled by some tiny factor (latent space just shrinks together)
        #      to avoid that, we use the detach() function, making sure that codebook and encoder outputs move individually and not just shrink together to something tiny
        loss_commitment = (x - x_rounded.detach()).square().mean() #only have gradient for encoder 
        loss_codebook   = (x.detach() - x_rounded).square().mean() #only have gradient for codebook

        #   5. the actual rounding we do: we take the difference between the rounded and the unrounded value and add it to the unrounded value
        #      the paper calls this "straight through estimator", as we just pass the gradient through the rounding operation (which is not differentiable)
        #      what essentially happens here is that we just take our x and substract some float without any gradient, similar to writing "x = x - 0.1":
        #      "(x - x_rounded).detach()" becomes just some number that we substract from x; this converts x to the same values as x_rounded, but the gradient is not passed through this operation
        #      i.e. we round, but keep the (then a bit inexact) gradient 
        x = x - (x - x_rounded).detach() #change x to x_rounded, but keep the gradient from x
        #   6. we reshape back to the original shape and return the values; we also return both losses and the indices of the closest codewords
        x = x.view(size_in)
        x = x.permute(0, 3, 1, 2).contiguous()
        return x, loss_codebook, loss_commitment, indices.view(x.size()[0], x.size()[2], x.size()[3])
    
    def encode(self, x):
        #1. encode
        res = self.residual_encoder(x)
        x = self.encoder(x)
        while x.size()[2] < res.size()[2]:
            res = self.reduce(res)
        x = x + res #self.upscale
        #2. re-shape and swap dimensions
        x = x.view(x.size()[0], self.SETTINGS.VQVAE_C, -1)
        x = x.transpose(1,2)
        #3. project down
        w = x.size()[2]
        x_out = None
        for i in range(0, SETTINGS.NUM_HEADS):
            out = self.down_proj[i](x[:,:,int(w/SETTINGS.NUM_HEADS*(i)):int(w/SETTINGS.NUM_HEADS*(i+1))])
            if x_out == None:
                x_out = out
            else:
                x_out = torch.cat((x_out, out), 2)
        x = x_out

        #4. quantise
        x = x.contiguous().view(x.size()[0], self.SETTINGS.VQVAE_D, -1, 1) #unfold, here just for quantisation
        if not SETTINGS.DO_QUANTISE:
            x_ = x.clone()
        x, loss_codebook, loss_commitment, indices = self.quantise_individually(x)
        
        if not SETTINGS.DO_QUANTISE:
            x = x_.clone()

        #don't add individual offsets, but use the global one
        x = x + self.offset[None, :, :, None]
        return x, loss_codebook, loss_commitment, indices

    def decode(self, x, w, h):
        x = x.view(x.size()[0], self.SETTINGS.VQVAE_D, -1) #fold back after quantisation; REDUNDANT!!! only for using only decoder
        #5. project up
        x_out = None
        blocksize = int(x.size()[2]/SETTINGS.NUM_HEADS)
        for i in range(0, SETTINGS.NUM_HEADS):
            out = self.up_proj[i](x[:,:,(blocksize*i):(blocksize * (i+1))])
            if x_out == None:
                x_out = out
            else:
                x_out = torch.cat((x_out, out), 2)
        x = x_out
        
        #6. reshape back
        x = x.transpose(1,2) #swap global information and local information back again (channels vs spatial dimensions)
        x = x.contiguous().view(x.size()[0], self.SETTINGS.VQVAE_C, int(w/(2 ** self.SETTINGS.DS_SPEED_FACTOR)), int(h/(2 ** self.SETTINGS.DS_SPEED_FACTOR)))
        #7. decode
        res = self.residual_decoder(x)
        x = self.decoder(x)
        while x.size()[2] > res.size()[2]:
            res = self.upscale(res)
        x = x + res
        return x

    def forward(self, x, dropout_mask=None):
        #encode, then quantise, then decode; pass the codebook loss and commitment loss through so we can use them for training
        w, h = x.size()[2], x.size()[3]
        x, loss_codebook, loss_commitment, indices = self.encode(x)
        
        x = x.view(x.size()[0], self.SETTINGS.VQVAE_D, -1) #fold back after quantisation
        if dropout_mask != None:
            x = x * dropout_mask[:,None,:]
        
        x = self.decode(x, w, h)

        return x, loss_codebook, loss_commitment, indices

In [ ]:
ae = BaseAE(SETTINGS, c_in=SETTINGS.INPUT_C)
#sanity check if everything works
for a, b in trainloader:
    ae(a)
    break

<h1>we somehow messed up our performance when transferring back from lightning...<br/>
will be fixed later, but at least it still runs I guess...</h1>

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
EPSILON = 0.000001

ae = BaseAE(SETTINGS, c_in=SETTINGS.INPUT_C).to(DEVICE)
optimiser = torch.optim.AdamW(ae.parameters(), lr=0.0002, weight_decay=0.01)

index_count = torch.zeros(SETTINGS.VQVAE_C, SETTINGS.VQVAE_K).to(DEVICE)
magnitudes = torch.zeros(SETTINGS.VQVAE_C, SETTINGS.VQVAE_K).to(DEVICE)

#just for tracking stuff
rec_loss = []
last_output = None
last_output = None

steps = 0
for epoch in range(0, 100):
    losses = [0.0, 0.0, 0.0]
    #train
    this_epoch = 0
    batchidx = 0
    epoch_start_time = time.time()
    print("*** Starting to train epoch ",epoch," ***")
    for data, _ in trainloader:
        optimiser.zero_grad()

        data = data.to(DEVICE)
        out, loss_codebook, loss_commitment, indices = ae(data)

        #######################
        ### actual training ###
        #######################
        if True:
            if SETTINGS.USE_DROPOUT:
                dropout_mask_binary = torch.randint(0, 3, (data.size()[0],))
                dropout_mask = torch.ones(data.size()[0], SETTINGS.VQVAE_C, device=data.device)
                for i in range(dropout_mask.size()[0]):
                    if dropout_mask_binary[i] == 0:
                        dropout_mask[i,torch.randint(1, SETTINGS.VQVAE_C, (1,)).item():] = 0.0
            else:
                dropout_mask = None
            
            outputs, loss_codebook, loss_commitment, indices = ae(data, dropout_mask)
            
            indices = indices[:,:,0]

            if True:
                onehot = torch.nn.functional.one_hot(indices.detach(), num_classes=SETTINGS.VQVAE_K)
                onehot = onehot.sum(dim=0)
                #now shape [256 x 512]
                index_count += onehot

            #MSE
            loss_reconstruction = (outputs - data).square()
            loss_reconstruction = loss_reconstruction.mean()
        
            loss_non_rec = (loss_codebook * 0.25 + loss_commitment)
            if not SETTINGS.DO_QUANTISE: #if we don't quantise, don't do anything
                loss_non_rec = loss_non_rec * 0.0
            loss = loss_reconstruction + loss_non_rec
        
        #######################
        #### loss tracking ####
        #######################
        if True:
            losses[0] += loss_reconstruction.item()
            losses[1] += loss_codebook.item()
            losses[2] += loss_commitment.item()
            
            rec_loss.append(loss_reconstruction.item())
            rec_loss = rec_loss[-100:]
            
            if this_epoch == 100:
                time_taken = time.time() - epoch_start_time
                time_per_it = time_taken / this_epoch
                print("\t\tProjected total time for this epoch: ",((SETTINGS.TRAIN_BATCHES+SETTINGS.TEST_BATCHES) * time_per_it)," seconds")
        
            if (last_output == None or (time.time() > last_output + 20.0)) and batchidx > 100:
                print("\t\t\t"+str(batchidx/SETTINGS.TRAIN_BATCHES*100)+"% done, current running mean: "+str(sum(rec_loss)/len(rec_loss)))
                print("\t\t\t--> "+str((time.time() - epoch_start_time) / (batchidx/SETTINGS.TRAIN_BATCHES) - (time.time() - epoch_start_time))+" seconds left until epoch is concluded")
                last_output = time.time()

        loss.backward()
        magnitudes += ae.codebook.grad.abs().mean(dim=2).detach()
        optimiser.step()

        steps += 1
        batchidx += 1

        #######################
        ### codebook resets ###
        #######################
        if steps % SETTINGS.NUM_TRAIN_BATCHES_UNTIL_RESET == 0:
            print("\t\t---> RE-ALIGNING FREQUENCIES")
            
            re_aligned_frequencies_total = []
            for frequency in range(0, SETTINGS.VQVAE_C):
                #1. while there exist codewords with magnitude = 0, re-initialise them
                re_aligned_frequencies = 0
                while True:
                    #find a codeword to re-distribute
                    unused_codeword = magnitudes[frequency].argmin()
                    if magnitudes[frequency, unused_codeword] > 0:
                        break

                    #find a codeword to split up
                    replacement_codeword = magnitudes[frequency].argmax()
                    if magnitudes[frequency, replacement_codeword] <= EPSILON:
                        break

                    ### torch.zeros(self.SETTINGS.VQVAE_C, self.SETTINGS.VQVAE_K, self.SETTINGS.VQVAE_D)
                    #re-distribute the codeword: find the direction with the highest magnitude and move the unused codeword in that direction
                    direction = (torch.rand(SETTINGS.VQVAE_D) * 2.0 - 1.0) * 0.0001
                    with torch.no_grad():
                        ae.codebook[frequency, unused_codeword] = ae.codebook[frequency, replacement_codeword].to(ae.codebook[frequency, unused_codeword].get_device()) + direction.to(ae.codebook[frequency, unused_codeword].get_device())

                    #set the replacement codeword/unused codeword to EPSILON to make sure that we don't use it again
                    magnitudes[frequency, replacement_codeword] = EPSILON
                    magnitudes[frequency, unused_codeword]      = EPSILON
                    re_aligned_frequencies += 1
                re_aligned_frequencies_total.append(re_aligned_frequencies)
                
                #set back to zero for next epoch (accumulate all over again)
                magnitudes[frequency] *= 0.0
            print("\t\tRe-aligned ",sum(re_aligned_frequencies_total)/len(re_aligned_frequencies_total)," codewords on average per codebook")
    
    print("*** Starting to test epoch ",epoch," ***")
    batchidx = 0

    ### we track this only for curiousity - all real measurements of the paper were done afterwards, on saved images with existing libaries
    test_psnr = 0.0
    with torch.no_grad():
        for data, _ in testloader:
            data = data.to(DEVICE)
            out, loss_codebook, loss_commitment, indices = ae(data)

            outputs, loss_codebook, loss_commitment, indices = ae(data, None)
            
            test_psnr += (-10.0 * torch.log10((outputs - data).square().mean(dim=3).mean(dim=2).mean(dim=1))).sum()

            indices = indices[:,:,0]

            #output images after every epoch
            if batchidx == 0:
                print("Examples from dataset:")
                save_image(torchvision.utils.make_grid(data.cpu().detach()).cpu().detach(), "outputs/"+SETTINGS.RUN_NAME+"/epoch_"+str(epoch)+"_testset_in.png")
                save_image(torchvision.utils.make_grid(outputs.cpu().detach()).cpu().detach(), "outputs/"+SETTINGS.RUN_NAME+"/epoch_"+str(epoch)+"_testset_out.png")
                imshow(torchvision.utils.make_grid(data.cpu().detach()).cpu().detach())
                imshow(torchvision.utils.make_grid(outputs.cpu().detach()).cpu().detach())

            batchidx += 1
    test_psnr /= len(testloader.dataset)

    losses[0] /= len(trainloader)
    losses[1] /= len(trainloader)
    losses[2] /= len(trainloader)

    print("*** RESULTS FOR EPOCH ",epoch," ***")
    print("AVG reconstruction loss: ",losses[0])
    print("AVG codebook loss: ",losses[1])
    print("AVG commitment loss: ",losses[2])
    print("Test PSNR: ",test_psnr.item())

    print("_____________________________\n\n\n")
    
#save for later use
torch.save(ae.state_dict(), "outputs/trained.net")